# Lab 1

data loading

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from ipywidgets import interact
import ipywidgets as widgets
from ipywidgets import interact, FloatSlider
from IPython.display import display, clear_output

In [2]:
file_path = 'data/Hurricane.xlsx'
do_data = pd.read_excel(file_path, sheet_name = 5).drop(['Piermont D.O. (ppm)'], axis = 1)
rainfall_data = pd.read_excel(file_path, sheet_name = 'Rainfall').drop(['Piermont  Rainfall Daily Accumulation (Inches)'], axis = 1)
turbidity_data = pd.read_excel(file_path, sheet_name = 'Turbidity').drop(['Piermont Turbidity in NTU'], axis = 1)


Data Cleaning

In [3]:
# merge the three datasets

data = do_data.merge(rainfall_data, on = "Date Time (ET)")
data = data.merge(turbidity_data, on = "Date Time (ET)")

# update column names 

data.columns = ['date', 'albany_DO', 'norrie_DO', 'albany_rainfall', 'norrie_rainfall', 'albany_turbidity', 'norrie_turbidity']

data['date'] = pd.to_datetime(data['date'])

data.set_index('date', inplace = True)

Multiple Linear Regression

In [4]:
# define predictors and the target variable
X = data[['albany_DO', 'albany_rainfall']]
y = data[['albany_turbidity']]

# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

# Create and fit the model
model = LinearRegression()
model.fit(X_train, y_train)

# predict and evaluate
y_pred = model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
rmse

r2 = r2_score(y_test, y_pred)
r2

0.4907389518457509

Create a widget to visualize different models

In [6]:
# Create a widget for selecting predictors
predictor_selector = widgets.SelectMultiple(
    options = data.columns,
    value = [data.columns[0]],
    description = 'Predictors'
)

# Create a dropdown for selecting the target variable
target_selector = widgets.Dropdown(
    options = data.columns,
    value = data.columns[1],
    description = 'Target'
)

# Button to evaluate the model
evaluate_button = widgets.Button(
    description = 'Evaluate'
)

# Output widget to display results
output = widgets.Output()

# Define the function to handle button clicks
def evaluate_model(b):
    with output:
        clear_output(wait = True) # Clear output of display area
        
        # Make sure the target is not in the predictors
        selected_predictors = [item for item in predictor_selector.value]
        if target_selector.value in selected_predictors:
            print("Target variable must not be in the predictors.")
            return
        
        # Define predictors and the target variable
        X = data[selected_predictors]
        y = data[target_selector.value]
        
        # Split the data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)
        
        # Create and fit the model
        model = LinearRegression()
        model.fit(X_train, y_train)
        
        #Predict and evaluate
        Y_pred = model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Display the values
        print(f"R^2: {r2:.4f}")
        print(f"MSE: {mse:.4f}")
        
# Display the widgets and connect the button to the function
display(predictor_selector, target_selector, evaluate_button, output)
evaluate_button.on_click(evaluate_model)




SelectMultiple(description='Predictors', index=(0,), options=('albany_DO', 'norrie_DO', 'albany_rainfall', 'no…

Dropdown(description='Target', index=1, options=('albany_DO', 'norrie_DO', 'albany_rainfall', 'norrie_rainfall…

Button(description='Evaluate', style=ButtonStyle())

Output()